## Date Handling of Input Files

> The data contains periods in CET/CEST format.
	

* we need to have start and end date in one consitent time zone
* we need to split the period into start and end date of the period 

"01/01/2015 00:00:00 - 01/01/2015 00:15:00"


## Setup

All the manipulations and plots in this notebook can be created with standard libraries such as matplotlib, statsmodels etc. 

In [2]:
# Main data packages. 
import numpy as np
import pandas as pd


## Import Data 

The data for this notebook was downloaded from the [meteoblue website](https://www.meteoblue.com/en/weather/archive/export/basel_switzerland_2661604) and consits of weather data for the city of Basel from 2008 till 2020. 

In [3]:
files = [
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_201412312300-201512312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_201512312300-201612312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_201612312300-201712312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_201712312300-201812312300 - 1.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_201712312300-201812312300 - 2.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_201812312300-201912312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_201912312300-202012312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_202012312300-202112312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_202112312300-202212312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_202212312300-202312312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_202312312300-202412312300.csv",
    "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_202412312300-202512312300.csv",
    # "../../data/total_gen_forecast_da/GUI_TOTAL_GENERATION_FORECAST_202512312300-202612312300.csv",
]

raw_df = pd.concat((pd.read_csv(f, delimiter=",") for f in files), ignore_index=True)


In [4]:
raw_df.shape  #(541275, 6)

(210384, 5)

In [5]:
raw_df.columns

Index(['MTU (CET/CEST)', 'Area', 'Generation Forecast (MW)',
       'Actual Generation (MW)', 'Scheduled Consumption (MW)'],
      dtype='object')

In [6]:
# Create working copy of dataframe
df_utc_op = raw_df.copy()
df_utc_op = df_utc_op[["MTU (CET/CEST)","Generation Forecast (MW)", "Actual Generation (MW)"]].rename(  #"Intraday (MW)","Current (MW)",
    columns={
        'MTU (CET/CEST)': 'period',  
        "Actual Generation (MW)": "gen_actual",
        "Generation Forecast (MW)": "gen_forecast_da",
    }
)
df_utc_op.head()

,period,gen_forecast_da,gen_actual
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,NaN,51471.98
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,NaN,51310.71
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,NaN,51446.23
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,NaN,50864.68
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,NaN,50247.45


In [7]:
df_utc_op.shape

(210384, 3)

In [8]:
# Split "period" into start/end strings
df_utc_op[["period_start", "period_end"]] = df_utc_op["period"].str.split(" - ", n=1, expand=True)
df_utc_op.head()

,period,gen_forecast_da,gen_actual,period_start,period_end
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,NaN,51471.98,01/01/2015 00:00:00,01/01/2015 00:15:00
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,NaN,51310.71,01/01/2015 00:15:00,01/01/2015 00:30:00
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,NaN,51446.23,01/01/2015 00:30:00,01/01/2015 00:45:00
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,NaN,50864.68,01/01/2015 00:45:00,01/01/2015 01:00:00
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,NaN,50247.45,01/01/2015 01:00:00,01/01/2015 01:15:00


In [9]:

def add_timezone_and_utc(df, col): #="period_start"):
    # State: start in CET
    tz_state = "CET"
    tz_list = []

    # Walk rows in order and flip state if original string contains CET/CEST
    for val in df[col].astype(str):
        if "CEST" in val:
            tz_state = "CEST"
        elif "CET" in val:
            tz_state = "CET"
        tz_list.append(tz_state)

    # Parse datetime from the column (remove any existing timezone text)
    base_dt = pd.to_datetime(
        df[col].astype(str).str.replace(r"\s*\(.*\)$", "", regex=True),
        dayfirst=True,
        errors="coerce"
    )

    # Add labeled version
    # df["period_start_labeled"] = (
    df[col + "_labeled"] = (
        base_dt.dt.strftime("%d/%m/%Y %H:%M:%S") + " (" + pd.Series(tz_list, index=df.index) + ")"
    )

    # Compute UTC by subtracting 1 or 2 hours depending on CET/CEST
    offset_hours = pd.Series(tz_list, index=df.index).map({"CET": 1, "CEST": 2})
    df[col + "_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")
    # df["tsp_start_utc"] = base_dt - pd.to_timedelta(offset_hours, unit="h")

    return df


In [10]:
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_start")
df_utc_op = add_timezone_and_utc(df_utc_op, col="period_end")

In [11]:
df_utc_op

,period,gen_forecast_da,gen_actual,period_start,period_end,period_start_labeled,period_start_utc,period_end_labeled,period_end_utc
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,NaN,51471.98,01/01/2015 00:00:00,01/01/2015 00:15:00,01/01/2015 00:00:00 (CET),2014-12-31 23:00:00,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,NaN,51310.71,01/01/2015 00:15:00,01/01/2015 00:30:00,01/01/2015 00:15:00 (CET),2014-12-31 23:15:00,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,NaN,51446.23,01/01/2015 00:30:00,01/01/2015 00:45:00,01/01/2015 00:30:00 (CET),2014-12-31 23:30:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,NaN,50864.68,01/01/2015 00:45:00,01/01/2015 01:00:00,01/01/2015 00:45:00 (CET),2014-12-31 23:45:00,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,NaN,50247.45,01/01/2015 01:00:00,01/01/2015 01:15:00,01/01/2015 01:00:00 (CET),2015-01-01 00:00:00,01/01/2015 01:15:00 (CET),2015-01-01 00:15:00
...,...,...,...,...,...,...,...,...,...
210379,31/12/2025 22:45:00 - 31/12/2025 23:00:00,57100.9,59243.58,31/12/2025 22:45:00,31/12/2025 23:00:00,31/12/2025 22:45:00 (CET),2025-12-31 21:45:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00
210380,31/12/2025 23:00:00 - 31/12/2025 23:15:00,56595.74,57595.91,31/12/2025 23:00:00,31/12/2025 23:15:00,31/12/2025 23:00:00 (CET),2025-12-31 22:00:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00
210381,31/12/2025 23:15:00 - 31/12/2025 23:30:00,56768.73,58613.15,31/12/2025 23:15:00,31/12/2025 23:30:00,31/12/2025 23:15:00 (CET),2025-12-31 22:15:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00
210382,31/12/2025 23:30:00 - 31/12/2025 23:45:00,57064.95,59518.66,31/12/2025 23:30:00,31/12/2025 23:45:00,31/12/2025 23:30:00 (CET),2025-12-31 22:30:00,31/12/2025 23:45:00 (CET),2025-12-31 22:45:00


In [12]:
df_utc_op.columns

Index(['period', 'gen_forecast_da', 'gen_actual', 'period_start', 'period_end',
       'period_start_labeled', 'period_start_utc', 'period_end_labeled',
       'period_end_utc'],
      dtype='object')

In [13]:
# Create working copy of dataframe
df_utc_op_temp = df_utc_op.copy()
# select only those values
# df_price = df_price[df_price["Sequence"] != "Sequence Sequence 2"]
 # drop columns
df_utc_op.drop(columns=['period_start', 'period_end', 'period_start_labeled', 'period_end_labeled'], inplace=True)

df_utc_op = df_utc_op.assign(
    date1=lambda x: x["period_start_utc"].dt.date,
    date=lambda x: pd.to_datetime(
        x["date1"],
        format="mixed",
        dayfirst=True,
        errors="coerce"
    ),
    year=lambda x: x["period_start_utc"].dt.year,
    month=lambda x: x["period_start_utc"].dt.month,
    #monthname=lambda x: x["period_start_utc"].dt.month_name(),
    day=lambda x: x["period_start_utc"].dt.day,
    dayofyear=lambda x: x["period_start_utc"].dt.dayofyear,
    hour=lambda x: x["period_start_utc"].dt.hour,
    #week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype(int),
    week=lambda x: x["period_start_utc"].dt.isocalendar().week.astype("Int64"),
    dayofweek=lambda x: x["period_start_utc"].dt.dayofweek,   # 0=Mon ... 6=Sun
    #dayname=lambda x: x["period_start_utc"].dt.day_name(),   

)
df_utc_op.drop(columns=['date1'], inplace=True)

df_utc_op.head()

,period,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,NaN,51471.98,2014-12-31 23:00:00,2014-12-31 23:15:00,2014-12-31,2014,12,31,365,23,1,2
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,NaN,51310.71,2014-12-31 23:15:00,2014-12-31 23:30:00,2014-12-31,2014,12,31,365,23,1,2
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,NaN,51446.23,2014-12-31 23:30:00,2014-12-31 23:45:00,2014-12-31,2014,12,31,365,23,1,2
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,NaN,50864.68,2014-12-31 23:45:00,2015-01-01 00:00:00,2014-12-31,2014,12,31,365,23,1,2
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,NaN,50247.45,2015-01-01 00:00:00,2015-01-01 00:15:00,2015-01-01,2015,1,1,1,0,1,3


In [14]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210384 entries, 0 to 210383
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            210384 non-null  object        
 1   gen_forecast_da   98635 non-null   object        
 2   gen_actual        210384 non-null  object        
 3   period_start_utc  210384 non-null  datetime64[ns]
 4   period_end_utc    210384 non-null  datetime64[ns]
 5   date              210384 non-null  datetime64[ns]
 6   year              210384 non-null  int32         
 7   month             210384 non-null  int32         
 8   day               210384 non-null  int32         
 9   dayofyear         210384 non-null  int32         
 10  hour              210384 non-null  int32         
 11  week              210384 non-null  Int64         
 12  dayofweek         210384 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), int32(6), object(3)
memory 

In [15]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["gen_forecast_da"].isna()]
rows_with_missing_price.tail(100)

,period,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
111649,26/09/2018 02:00:00 - 26/09/2018 03:00:00,NaN,58581.42,2018-09-26 00:00:00,2018-09-26 01:00:00,2018-09-26,2018,9,26,269,0,39,2
111650,26/09/2018 03:00:00 - 26/09/2018 04:00:00,NaN,58433.84,2018-09-26 01:00:00,2018-09-26 02:00:00,2018-09-26,2018,9,26,269,1,39,2
111651,26/09/2018 04:00:00 - 26/09/2018 05:00:00,NaN,59672.93,2018-09-26 02:00:00,2018-09-26 03:00:00,2018-09-26,2018,9,26,269,2,39,2
111652,26/09/2018 05:00:00 - 26/09/2018 06:00:00,NaN,62921.38,2018-09-26 03:00:00,2018-09-26 04:00:00,2018-09-26,2018,9,26,269,3,39,2
111653,26/09/2018 06:00:00 - 26/09/2018 07:00:00,NaN,70936.31,2018-09-26 04:00:00,2018-09-26 05:00:00,2018-09-26,2018,9,26,269,4,39,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173759,27/10/2024 00:00:00 - 27/10/2024 01:00:00,NaN,39731.12,2024-10-26 22:00:00,2024-10-26 23:00:00,2024-10-26,2024,10,26,300,22,43,5
203948,26/10/2025 00:00:00 - 26/10/2025 00:15:00,NaN,53272.4,2025-10-25 22:00:00,2025-10-25 22:15:00,2025-10-25,2025,10,25,298,22,43,5
203949,26/10/2025 00:15:00 - 26/10/2025 00:30:00,NaN,53053.62,2025-10-25 22:15:00,2025-10-25 22:30:00,2025-10-25,2025,10,25,298,22,43,5
203950,26/10/2025 00:30:00 - 26/10/2025 00:45:00,NaN,53080.87,2025-10-25 22:30:00,2025-10-25 22:45:00,2025-10-25,2025,10,25,298,22,43,5


In [16]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["gen_actual"].isna()]
rows_with_missing_price

,period,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek


In [17]:
# Convert to numeric, turn non-numeric strings into NaN
df_utc_op["gen_forecast_da"] = pd.to_numeric(df_utc_op["gen_forecast_da"], errors="coerce")
df_utc_op["gen_actual"] = pd.to_numeric(df_utc_op["gen_actual"], errors="coerce")

rows_with_missing_price = df_utc_op[df_utc_op["gen_forecast_da"].isna()]
rows_with_missing_price

,period,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,NaN,51471.98,2014-12-31 23:00:00,2014-12-31 23:15:00,2014-12-31,2014,12,31,365,23,1,2
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,NaN,51310.71,2014-12-31 23:15:00,2014-12-31 23:30:00,2014-12-31,2014,12,31,365,23,1,2
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,NaN,51446.23,2014-12-31 23:30:00,2014-12-31 23:45:00,2014-12-31,2014,12,31,365,23,1,2
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,NaN,50864.68,2014-12-31 23:45:00,2015-01-01 00:00:00,2014-12-31,2014,12,31,365,23,1,2
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,NaN,50247.45,2015-01-01 00:00:00,2015-01-01 00:15:00,2015-01-01,2015,1,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173759,27/10/2024 00:00:00 - 27/10/2024 01:00:00,NaN,39731.12,2024-10-26 22:00:00,2024-10-26 23:00:00,2024-10-26,2024,10,26,300,22,43,5
203948,26/10/2025 00:00:00 - 26/10/2025 00:15:00,NaN,53272.40,2025-10-25 22:00:00,2025-10-25 22:15:00,2025-10-25,2025,10,25,298,22,43,5
203949,26/10/2025 00:15:00 - 26/10/2025 00:30:00,NaN,53053.62,2025-10-25 22:15:00,2025-10-25 22:30:00,2025-10-25,2025,10,25,298,22,43,5
203950,26/10/2025 00:30:00 - 26/10/2025 00:45:00,NaN,53080.87,2025-10-25 22:30:00,2025-10-25 22:45:00,2025-10-25,2025,10,25,298,22,43,5


In [18]:
df_utc_op.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210384 entries, 0 to 210383
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   period            210384 non-null  object        
 1   gen_forecast_da   89875 non-null   float64       
 2   gen_actual        201624 non-null  float64       
 3   period_start_utc  210384 non-null  datetime64[ns]
 4   period_end_utc    210384 non-null  datetime64[ns]
 5   date              210384 non-null  datetime64[ns]
 6   year              210384 non-null  int32         
 7   month             210384 non-null  int32         
 8   day               210384 non-null  int32         
 9   dayofyear         210384 non-null  int32         
 10  hour              210384 non-null  int32         
 11  week              210384 non-null  Int64         
 12  dayofweek         210384 non-null  int32         
dtypes: Int64(1), datetime64[ns](3), float64(2), int32(6), objec

In [19]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "hour", "week", "dayofweek"
]

df_utc_h = (
    df_utc_op
    .groupby(group_cols, as_index=False)
    .agg(
        gen_forecast_da=("gen_forecast_da", "mean"),
        gen_actual=("gen_actual", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_hour=("year", "size"),   # count rows per group
    )
)
df_utc_h.head()

,date,year,month,day,dayofyear,hour,week,dayofweek,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,c_by_hour
0,2014-12-31,2014,12,31,365,23,1,2,NaN,51273.4000,2014-12-31 23:00:00,2015-01-01 00:00:00,4
1,2015-01-01,2015,1,1,1,0,1,3,NaN,49707.0550,2015-01-01 00:00:00,2015-01-01 01:00:00,4
2,2015-01-01,2015,1,1,1,1,1,3,NaN,48784.9325,2015-01-01 01:00:00,2015-01-01 02:00:00,4
3,2015-01-01,2015,1,1,1,2,1,3,NaN,47937.0925,2015-01-01 02:00:00,2015-01-01 03:00:00,4
4,2015-01-01,2015,1,1,1,3,1,3,NaN,48116.0850,2015-01-01 03:00:00,2015-01-01 04:00:00,4


In [20]:
df_utc_h.tail()

,date,year,month,day,dayofyear,hour,week,dayofweek,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,c_by_hour
96427,2025-12-31,2025,12,31,365,18,1,2,58900.9800,60292.3475,2025-12-31 18:00:00,2025-12-31 19:00:00,4
96428,2025-12-31,2025,12,31,365,19,1,2,58562.3375,58865.2100,2025-12-31 19:00:00,2025-12-31 20:00:00,4
96429,2025-12-31,2025,12,31,365,20,1,2,57650.8775,59417.6050,2025-12-31 20:00:00,2025-12-31 21:00:00,4
96430,2025-12-31,2025,12,31,365,21,1,2,56819.4225,59089.8875,2025-12-31 21:00:00,2025-12-31 22:00:00,4
96431,2025-12-31,2025,12,31,365,22,1,2,56897.7500,58908.9675,2025-12-31 22:00:00,2025-12-31 23:00:00,4


In [21]:
df_utc_h["c_by_hour"].value_counts()

c_by_hour
1    52608
4    35064
2     8760
Name: count, dtype: int64

In [22]:
df_utc_h[df_utc_h["c_by_hour"] > 4]

,date,year,month,day,dayofyear,hour,week,dayofweek,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,c_by_hour


In [23]:
group_cols = [
    "date", "year", "month", "day", "dayofyear", "week", "dayofweek"
]

df_utc_d = (
    df_utc_h
    .groupby(group_cols, as_index=False)
    .agg(
        gen_forecast_da=("gen_forecast_da", "mean"),
        gen_actual=("gen_actual", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_day=("year", "size"),   # count rows per group
    )
)
df_utc_d.head()

,date,year,month,day,dayofyear,week,dayofweek,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,c_by_day
0,2014-12-31,2014,12,31,365,1,2,NaN,51273.400000,2014-12-31 23:00:00,2015-01-01,1
1,2015-01-01,2015,1,1,1,1,3,NaN,52918.358333,2015-01-01 00:00:00,2015-01-02,24
2,2015-01-02,2015,1,2,2,1,4,NaN,62965.999375,2015-01-02 00:00:00,2015-01-03,24
3,2015-01-03,2015,1,3,3,1,5,NaN,60103.508646,2015-01-03 00:00:00,2015-01-04,24
4,2015-01-04,2015,1,4,4,1,6,NaN,57891.244896,2015-01-04 00:00:00,2015-01-05,24


In [24]:
df_utc_d.tail()

,date,year,month,day,dayofyear,week,dayofweek,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,c_by_day
4014,2025-12-27,2025,12,27,361,52,5,44299.660104,46320.834896,2025-12-27,2025-12-28 00:00:00,24
4015,2025-12-28,2025,12,28,362,52,6,41008.039167,42641.979479,2025-12-28,2025-12-29 00:00:00,24
4016,2025-12-29,2025,12,29,363,1,0,52262.052813,54433.414271,2025-12-29,2025-12-30 00:00:00,24
4017,2025-12-30,2025,12,30,364,1,1,50291.344688,51543.761563,2025-12-30,2025-12-31 00:00:00,24
4018,2025-12-31,2025,12,31,365,1,2,55791.667935,58980.592065,2025-12-31,2025-12-31 23:00:00,23


In [25]:
df_utc_d["c_by_day"].value_counts()

c_by_day
24    4017
1        1
23       1
Name: count, dtype: int64

In [26]:
group_cols = [
    "year"
]

df_utc_y = (
    df_utc_d
    .groupby(group_cols, as_index=False)
    .agg(
        gen_forecast_da=("gen_forecast_da", "mean"),
        gen_actual=("gen_actual", "mean"),
        period_start_utc=("period_start_utc", "min"),
        period_end_utc=("period_end_utc", "max"),
        c_by_year=("year", "size"),   # count rows per group
    )
)
df_utc_y.head(20)

,year,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,c_by_year
0,2014,NaN,51273.400000,2014-12-31 23:00:00,2015-01-01 00:00:00,1
1,2015,NaN,63289.078911,2015-01-01 00:00:00,2016-01-01 00:00:00,365
2,2016,NaN,64631.952633,2016-01-01 00:00:00,2017-01-01 00:00:00,366
3,2017,NaN,65548.891528,2017-01-01 00:00:00,2018-01-01 00:00:00,365
4,2018,61596.309526,67156.336949,2018-01-01 00:00:00,2019-01-01 00:00:00,365
5,2019,56620.563581,59856.600209,2019-01-01 00:00:00,2020-01-01 00:00:00,365
6,2020,51859.445079,57466.457105,2020-01-01 00:00:00,2021-01-01 00:00:00,366
7,2021,55005.675898,58504.799119,2021-01-01 00:00:00,2022-01-01 00:00:00,365
8,2022,56563.889325,57206.379349,2022-01-01 00:00:00,2023-01-01 00:00:00,365
9,2023,50879.359164,51906.517608,2023-01-01 00:00:00,2024-01-01 00:00:00,365


In [27]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["gen_forecast_da"].isna()]
rows_with_missing_price

,period,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
0,01/01/2015 00:00:00 - 01/01/2015 00:15:00,NaN,51471.98,2014-12-31 23:00:00,2014-12-31 23:15:00,2014-12-31,2014,12,31,365,23,1,2
1,01/01/2015 00:15:00 - 01/01/2015 00:30:00,NaN,51310.71,2014-12-31 23:15:00,2014-12-31 23:30:00,2014-12-31,2014,12,31,365,23,1,2
2,01/01/2015 00:30:00 - 01/01/2015 00:45:00,NaN,51446.23,2014-12-31 23:30:00,2014-12-31 23:45:00,2014-12-31,2014,12,31,365,23,1,2
3,01/01/2015 00:45:00 - 01/01/2015 01:00:00,NaN,50864.68,2014-12-31 23:45:00,2015-01-01 00:00:00,2014-12-31,2014,12,31,365,23,1,2
4,01/01/2015 01:00:00 - 01/01/2015 01:15:00,NaN,50247.45,2015-01-01 00:00:00,2015-01-01 00:15:00,2015-01-01,2015,1,1,1,0,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
173759,27/10/2024 00:00:00 - 27/10/2024 01:00:00,NaN,39731.12,2024-10-26 22:00:00,2024-10-26 23:00:00,2024-10-26,2024,10,26,300,22,43,5
203948,26/10/2025 00:00:00 - 26/10/2025 00:15:00,NaN,53272.40,2025-10-25 22:00:00,2025-10-25 22:15:00,2025-10-25,2025,10,25,298,22,43,5
203949,26/10/2025 00:15:00 - 26/10/2025 00:30:00,NaN,53053.62,2025-10-25 22:15:00,2025-10-25 22:30:00,2025-10-25,2025,10,25,298,22,43,5
203950,26/10/2025 00:30:00 - 26/10/2025 00:45:00,NaN,53080.87,2025-10-25 22:30:00,2025-10-25 22:45:00,2025-10-25,2025,10,25,298,22,43,5


In [28]:
# Check NaN only in one column ('price')
rows_with_missing_price = df_utc_op[df_utc_op["gen_actual"].isna()]
rows_with_missing_price

,period,gen_forecast_da,gen_actual,period_start_utc,period_end_utc,date,year,month,day,dayofyear,hour,week,dayofweek
111767,01/10/2018 00:00:00 - 01/10/2018 01:00:00,NaN,NaN,2018-09-30 22:00:00,2018-09-30 23:00:00,2018-09-30,2018,9,30,273,22,39,6
111768,01/10/2018 01:00:00 - 01/10/2018 02:00:00,NaN,NaN,2018-09-30 23:00:00,2018-10-01 00:00:00,2018-09-30,2018,9,30,273,23,39,6
111769,01/10/2018 02:00:00 - 01/10/2018 03:00:00,NaN,NaN,2018-10-01 00:00:00,2018-10-01 01:00:00,2018-10-01,2018,10,1,274,0,40,0
111770,01/10/2018 03:00:00 - 01/10/2018 04:00:00,NaN,NaN,2018-10-01 01:00:00,2018-10-01 02:00:00,2018-10-01,2018,10,1,274,1,40,0
111771,01/10/2018 04:00:00 - 01/10/2018 05:00:00,NaN,NaN,2018-10-01 02:00:00,2018-10-01 03:00:00,2018-10-01,2018,10,1,274,2,40,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120522,30/09/2018 19:00:00 - 30/09/2018 20:00:00,NaN,NaN,2018-09-30 17:00:00,2018-09-30 18:00:00,2018-09-30,2018,9,30,273,17,39,6
120523,30/09/2018 20:00:00 - 30/09/2018 21:00:00,NaN,NaN,2018-09-30 18:00:00,2018-09-30 19:00:00,2018-09-30,2018,9,30,273,18,39,6
120524,30/09/2018 21:00:00 - 30/09/2018 22:00:00,NaN,NaN,2018-09-30 19:00:00,2018-09-30 20:00:00,2018-09-30,2018,9,30,273,19,39,6
120525,30/09/2018 22:00:00 - 30/09/2018 23:00:00,NaN,NaN,2018-09-30 20:00:00,2018-09-30 21:00:00,2018-09-30,2018,9,30,273,20,39,6


## saving our files by hour

In [29]:
df_utc_h.to_csv("../../data_cleaned/by_source/06_TOTAL_GEN.csv", index=False)